In [ ]:
import glob
import json
import numpy as np
import pandas as pd
import os
from scipy.io import loadmat 
from keras.callbacks import CSVLogger
import random
import shutil 
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications import DenseNet121,MobileNet,InceptionV3,EfficientNetB1,VGG19,ResNet101
from keras.models import Model
from keras.optimizers import Adam
from keras.models import load_model
from keras.layers import Input, Dropout, Dense, GlobalAveragePooling2D, BatchNormalization
from keras.callbacks import ModelCheckpoint ,ReduceLROnPlateau ,EarlyStopping ,Callback


In [ ]:
DATA_PATH = './data'
ORIGINAL_PATH = '../input/102flowers/data/data'
# LABELS_PATH = '../input/102flowers/imagelabels.mat'
# mat = loadmat(LABELS_PATH)
# labels = mat['labels'][0]

In [ ]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH)
os.mkdir(DATA_PATH)
os.mkdir(DATA_PATH + '/' + 'train')
os.mkdir(DATA_PATH + '/' + 'validation')
os.mkdir(DATA_PATH + '/' + 'test')
split_dict = {}
for label_dir in os.listdir(ORIGINAL_PATH):
    imgs_in_label = len(os.listdir(ORIGINAL_PATH+'/'+label_dir))
    all_indxs_set = set(np.arange(imgs_in_label))
    train = set(random.sample(all_indxs_set, int(0.5*imgs_in_label)))
    validation = set(random.sample(all_indxs_set-train, int(0.25*imgs_in_label)))
    test = all_indxs_set-train-validation
    split_dict[label_dir] = {"train": [int(x) for x in list(train)],
                             "validation":[int(x) for x in list(validation)],
                             "test":[int(x) for x in list(test)]}
    i=0
    for filename in os.listdir(ORIGINAL_PATH+'/'+label_dir):
        if i in train:
            target = DATA_PATH + '/train/' + str(label_dir)
        elif i in validation:
            target = DATA_PATH + '/validation/' + str(label_dir)
        else:
            target = DATA_PATH + '/test/' + str(label_dir)
        if not os.path.exists(target):
            os.mkdir(target)
        shutil.copyfile(ORIGINAL_PATH+'/'+label_dir+'/'+filename, target+'/'+filename)
        i+=1
        
with open('data_split.json', 'w') as outfile:
    json.dump(split_dict, outfile)

In [ ]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
NUM_CLASSES=102

In [ ]:
def get_model():
    model = InceptionV3(include_top=False, weights='imagenet',input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    model.trainable = False
    for layer in model.layers:
        layer.trainable = False

    # Rebuild top
    x = GlobalAveragePooling2D()(model.output)
    x= BatchNormalization()(x)
#     x=Dense(2048,activation='relu')(x)
#     x= BatchNormalization()(x)
#     x = Dropout(0.4)(x)
#     x=Dense(1024,activation='relu')(x)
#     x= BatchNormalization()(x)
#     x=Dropout(0.4)(x) 
#     x=Dense(1024,activation='relu')(x)
#     x= BatchNormalization()(x)
#     x=Dropout(0.3)(x) 
    x=Dense(256,activation='relu')(x)
    x= BatchNormalization()(x)
    outputs = Dense(NUM_CLASSES, activation="softmax")(x)
    # Compile
    model = Model(model.inputs, outputs)
    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
        )
    return model

In [ ]:
# # ~93% 

# def get_model():
#     model = MobileNet(include_top=False, weights='imagenet',input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
#     model.trainable = False
#     for layer in model.layers:
#         layer.trainable = False

#     # Rebuild top
#     x = GlobalAveragePooling2D()(model.output)
# #     x= BatchNormalization()(x)
#     x=Dense(2048,activation='relu')(x)
#     x= BatchNormalization()(x)
# #     x = Dropout(0.3)(x)
#     x=Dense(1024,activation='relu')(x)
#     x= BatchNormalization()(x)
#     x=Dense(1024,activation='relu')(x)
#     x= BatchNormalization()(x)
#     x=Dropout(0.2)(x) 
#     x=Dense(512,activation='relu')(x)
#     x=Dropout(0.2)(x) 
#     x= BatchNormalization()(x)
#     x=Dropout(0.1)(x) 
#     x=Dense(256,activation='relu')(x)
#     x= BatchNormalization()(x)
#     outputs = Dense(NUM_CLASSES, activation="softmax")(x)
#     # Compile
#     model = Model(model.inputs, outputs)
#     optimizer = Adam(learning_rate=0.001)
#     model.compile(
#         optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
#         )
#     return model


In [ ]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = True

    optimizer = Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
batch_size = 64

train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
 width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
 horizontal_flip=True,vertical_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        DATA_PATH+'/train',  
        target_size=(IMG_HEIGHT, IMG_WIDTH),  
        batch_size=batch_size) 

validation_generator = test_datagen.flow_from_directory(
        DATA_PATH+'/validation',
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=batch_size)

test_generator = test_datagen.flow_from_directory(
        DATA_PATH+'/test',
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=batch_size)



In [ ]:
data = []     
labels = []   
max_iter = 5 
i = 0
for d, l in train_generator:
    data.append(d)
    labels.append(l)
    i += 1
    if i == max_iter:
        break
print(l[0])

In [ ]:
def plot_graphs(history,eval_test):
    acc_test = []
    loss_test = []
    for i in range(len(eval_test)):
        result = eval_test[i]
        loss_test.append(result[0])
        acc_test.append(result[1])
    
    x = np.arange(len(history['loss']))
    
    f = plt.figure()
    f.set_figwidth(8)
    f.set_figheight(6)
    
    
    plt.plot(x, history['loss'], label='Training',linewidth=2)
    plt.plot(x, history['val_loss'], label='Validation',linewidth=2)
    plt.plot(x, loss_test, label='Test',linewidth=2)



    plt.xlabel('Ephoc')
    plt.ylabel('Loss')
    plt.title('Results')

    plt.legend()
    plt.show()
    
    f = plt.figure()
    f.set_figwidth(8)
    f.set_figheight(6)
    
    plt.plot(x, history['accuracy'], label='Training',linewidth=2)
    plt.plot(x, history['val_accuracy'], label='Validation',linewidth=2)
    plt.plot(x, acc_test, label='Test',linewidth=2)
    


    plt.xlabel('Ephoc')
    plt.ylabel('Accuracy')
    plt.title('Results')
    
    plt.legend()
    plt.show()


In [ ]:
eval_results_1st , eval_results_2nd = [] , []
class EvaluationCallback1st(Callback):
    def on_epoch_end(self, epoch, logs=None):
        eval_results = self.model.evaluate_generator(test_generator)
        eval_results_1st.append(eval_results)
        
class EvaluationCallback2nd(Callback):
    def on_epoch_end(self, epoch, logs=None):
        eval_results = self.model.evaluate_generator(test_generator)
        eval_results_2nd.append(eval_results)

In [ ]:
NUM_EPOCHS = 30 #20
mc = ModelCheckpoint('1st_best_model1.h5', monitor=["val_loss"], verbose=1, mode='min')
csv_logger = CSVLogger('1st_training.log', separator=',', append=False)
# es = EarlyStopping(monitor = 'val_loss',mode='min',patience=2)
rlrop = ReduceLROnPlateau(patience=2)
eval_test = EvaluationCallback1st()
model = get_model()
history = model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8,callbacks=[eval_test,mc,csv_logger,rlrop] ,
                                           shuffle=True, validation_data = validation_generator)


In [ ]:
plot_graphs(history.history,eval_results_1st)

In [ ]:
model = load_model('1st_best_model.h5')

eval_results = model.evaluate_generator(test_generator)

for metric, value in zip(model.metrics_names, eval_results):
    print(metric + ': {:.4f}'.format(value))

In [ ]:
with open('eval_results_1st.json','w') as f:
     json.dump(eval_results_1st, f)

In [ ]:
unfreeze_model(model)
mc = ModelCheckpoint('2nd_best_model.h5', monitor=["val_loss"], verbose=1, mode='min')
csv_logger = CSVLogger('2nd_training.log', separator=',', append=False)
es = EarlyStopping(monitor = 'val_loss',mode='min',patience=4)
eval_test = EvaluationCallback2nd()
NUM_EPOCHS = 60 
rlrop = ReduceLROnPlateau(patience=2)
history = model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, callbacks=[eval_test,es,mc,csv_logger,rlrop],
    shuffle=True, validation_data = validation_generator)

In [ ]:
plot_graphs(history.history,eval_results_2nd)

In [ ]:

eval_results = model.evaluate_generator(test_generator)

for metric, value in zip(model.metrics_names, eval_results):
    print(metric + ': {:.4f}'.format(value))

In [ ]:
with open('eval_results_2nd.json','w') as f:
     json.dump(eval_results_2nd, f)